In [1]:
import gc
from all_stand_var import conv_dict, lab_cols, vent_cols3
from all_own_funct import memory_downscale,memory_upscale
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from seaborn import heatmap
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import kurtosis
from matplotlib.backends.backend_pdf import PdfPages
import keras.backend as K
from RNN_LTSM_CHD import return_loaded_model
import pickle
import locale
import LR_build_CHD as pp
import tables
import pickle

from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix


locale.setlocale(locale.LC_ALL, 'fr_FR')
# Set ouput_folder
output_folder = os.path.join(os.getcwd(), 'Results_LR_RF_CHD_v3','12u_result')

    
# Load in files from output folder
f = open(os.path.join(output_folder,'ran_for.sav'), 'rb')
best_rf=pickle.load(f)
f.close()

f = open(os.path.join(output_folder,'log_reg.sav'), 'rb')
best_lr=pickle.load(f)
f.close()

f = open(os.path.join(output_folder,'x_train.txt'), 'rb')
X_TRAIN=pickle.load(f)
f.close()

f = open(os.path.join(output_folder,'x_test.txt'), 'rb')
X_TEST=pickle.load(f)
f.close()
f = open(os.path.join(output_folder,'y_train.txt'), 'rb')
Y_TRAIN=pickle.load(f)
f.close()
f = open(os.path.join(output_folder,'y_test.txt'), 'rb')
Y_TEST=pickle.load(f)
f.close()

C:\Users\berend\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# Calculate feature importance for logistic regression
clf=best_lr
float_columns = list(X_TRAIN.columns)

# create pdf with all figures
try:
    pdf = PdfPages(os.path.join(output_folder,f"Figures_importance_prob.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"Figures_importance_prob.pdf"))

# calculate coefficients
try:
    coefs = clf.coef_.flatten()
except:
    coefs=clf.feature_importances_

# Zip coefficients and names together and make a DataFrame
zipped = zip(float_columns, coefs)
df = pd.DataFrame(zipped, columns=["feature", "value"])
# Sort the features by the absolute value of their coefficient
df["abs_value"] = df["value"].apply(lambda x: abs(x))
df["colors"] = df["value"].apply(lambda x: "green" if x > 0 else "red")
df = df.sort_values("abs_value", ascending=False)


import seaborn as sns
fig, ax = plt.subplots(1, 1,figsize=(14,16))
sns.barplot(x="feature",
            y="value",
            data=df.head(10),
            palette=df.head(10)["colors"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=65, fontsize=18)
ax.set_title("Top 10 Features for best logistic regression model", fontsize=18)
ax.set_ylabel("Coef", fontsize=18)
ax.set_xlabel("Feature Name", fontsize=18)
fig=plt.gcf()
plt.savefig(os.path.join(output_folder,'Feature_importance.png'), format='png',
             dpi=300, facecolor='white', transparent=True, bbox_inches='tight')
pdf.savefig(fig)
plt.close(fig) 


['pat_weight_act_mean', 'pat_weight_act_std', 'pat_weight_act_lin_reg_coef', 'mon_etco2_mean', 'mon_etco2_std', 'mon_etco2_lin_reg_coef', 'mon_hr_mean', 'mon_hr_std', 'mon_hr_lin_reg_coef', 'mon_ibp_mean_mean', 'mon_ibp_mean_std', 'mon_ibp_mean_lin_reg_coef', 'mon_rr_mean', 'mon_rr_std', 'mon_rr_lin_reg_coef', 'mon_sat_mean', 'mon_sat_std', 'mon_sat_lin_reg_coef', 'vent_m_fio2_mean', 'vent_m_fio2_std', 'vent_m_fio2_lin_reg_coef', 'vent_m_peep_mean', 'vent_m_peep_std', 'vent_m_peep_lin_reg_coef', 'vent_m_ppeak_mean', 'vent_m_ppeak_std', 'vent_m_ppeak_lin_reg_coef', 'vent_m_rr_mean', 'vent_m_rr_std', 'vent_m_rr_lin_reg_coef', 'vent_m_tv_exp_mean', 'vent_m_tv_exp_std', 'vent_m_tv_exp_lin_reg_coef', 'Age_mean', 'Age_std', 'Age_lin_reg_coef', 'Diagnose_mean', 'Diagnose_std', 'Diagnose_lin_reg_coef', 'mis_mean', 'mis_std', 'mis_lin_reg_coef']


In [4]:
# Calculate feature importance for Random forest
clf=best_rf

try:
    coefs = clf.coef_.flatten()
except:
    coefs=clf.feature_importances_

# Zip coefficients and names together and make a DataFrame
zipped = zip(float_columns, coefs)
df = pd.DataFrame(zipped, columns=["feature", "value"])
# Sort the features by the absolute value of their coefficient
df["abs_value"] = df["value"].apply(lambda x: abs(x))
df["colors"] = df["value"].apply(lambda x: "green" if x > 0 else "red")
df = df.sort_values("abs_value", ascending=False)


import seaborn as sns
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
sns.barplot(x="feature",
            y="value",
            data=df.head(10),
            palette=df.head(10)["colors"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=65, fontsize=14)
ax.set_title("Top 10 Features for best random forest model", fontsize=18)
ax.set_ylabel("Coef", fontsize=18)
ax.set_xlabel("Feature Name", fontsize=18)
print(df["feature"].head(20).tolist())
plt.tight_layout()
fig=plt.gcf()
plt.savefig(os.path.join(output_folder,'Feature_importance_rf.png'), format='png',
             dpi=300, facecolor='white', transparent=True, bbox_inches='tight')
pdf.savefig(fig)
plt.close(fig) 


['mon_hr_mean', 'pat_weight_act_mean', 'vent_m_peep_lin_reg_coef', 'vent_m_rr_mean', 'mon_etco2_mean', 'Age_mean', 'mon_rr_mean', 'mon_hr_std', 'vent_m_tv_exp_mean', 'vent_m_ppeak_mean', 'vent_m_peep_mean', 'mon_ibp_mean_std', 'vent_m_tv_exp_std', 'mon_ibp_mean_mean', 'vent_m_ppeak_lin_reg_coef', 'vent_m_ppeak_std', 'vent_m_fio2_mean', 'vent_m_rr_std', 'mon_rr_lin_reg_coef', 'vent_m_fio2_lin_reg_coef']


In [ ]:
# create the prediction probability for logistic regression
# probability estimate
prediction=best_lr.predict_proba(X_TEST)[:,1]
true_0 = (Y_TEST == 0.0)
true_1 = (Y_TEST == 1.0)
df_test=pd.DataFrame()
df_test['prob']=best_lr.predict_proba(X_TEST)[:,1]
df_test['true']=Y_TEST

plt.figure(figsize=(15,7))
plt.hist(df_test['prob'].loc[df_test['true']==0], bins=50, label='Negatives')
plt.hist(df_test['prob'].loc[df_test['true']==1], bins=50, label='Positives', alpha=0.7, color='r')
plt.xlabel('Probability of being Positive Class', fontsize=25)
plt.ylabel('Number of records in each bucket', fontsize=25)
plt.title('Probability distribution of best LR')
plt.legend(fontsize=15)
plt.tick_params(axis='both', labelsize=25, pad=5)
fig=plt.gcf()
pdf.savefig(fig)
plt.show(fig)
plt.close(fig) 

In [ ]:
# create the prediction probability for logistic regression
# probability estimate
prediction=best_rf.predict_proba(X_TRAIN)[:,1]
true_0 = (Y_TEST == 0.0)
true_1 = (Y_TEST == 1.0)
df_test=pd.DataFrame()
df_test['prob']=best_rf.predict_proba(X_TEST)[:,1]
df_test['true']=Y_TEST

plt.figure(figsize=(15,7))
plt.hist(df_test['prob'].loc[df_test['true']==0], bins=50, label='Negatives')
plt.hist(df_test['prob'].loc[df_test['true']==1], bins=50, label='Positives', alpha=0.8, color='r')
plt.xlabel('Probability of being Positive Class', fontsize=25)
plt.ylabel('Number of records in each bucket', fontsize=25)
plt.title('Probability distribution of best RF')
plt.legend(fontsize=15)
plt.tick_params(axis='both', labelsize=25, pad=5)
fig=plt.gcf()
pdf.savefig(fig)
plt.show(fig)
plt.close(fig)
pdf.close()